In [9]:
import pandas as pd
import numpy as np
import string

In [17]:
test_values=pd.read_csv('data/Test Set Values.csv')
train_values=pd.read_csv('data/Training Set Values.csv')

In [110]:
# Getting columns
extraction_type=train_values['extraction_type']
extraction_type_group=train_values['extraction_type_group']
quantity=train_values['quantity']
quantity_group=train_values['quantity_group']
source=train_values['source']
source_class=train_values['source_class']
source_type=train_values['source_type']
wpt=train_values['waterpoint_type']
wptg=train_values['waterpoint_type_group']

In [18]:
# For converting excel columns to numerical index (from 0)
def col2num(col):
    num=0
    for c in col:
        if c in string.ascii_letters:
            num = num * 26 + (ord(c.upper()) - ord('A'))
    return num

In [105]:
# Make a function to check unique values of columns:
def check_same(a, b):
    print('{} has {} unique values \n'.format(a.name, len(a.unique())))
    print('{} has {} unique values \n'.format(b.name, len(b.unique())))
    sym_diff=set(a)^set(b)
    print('{} elements in {} or {} but not both: \n {}'.format(len(sym_diff), a.name, b.name, sym_diff))
    check=a==b
    check_false=len(check[check==False])
    print('{pct:.{digits}f}% of elements are the same'.format(pct=100-check_false/len(check), digits=2))

In [111]:
check_same(extraction_type,extraction_type_group)

extraction_type has 18 unique values 

extraction_type_group has 13 unique values 

13 elements in extraction_type or extraction_type_group but not both: 
 {'walimi', 'wind-powered', 'rope pump', 'other motorpump', 'other - swn 81', 'ksb', 'windmill', 'other - rope pump', 'other - play pump', 'cemo', 'climax', 'other - mkulima/shinyanga', 'other handpump'}
99.96% of elements are the same


In [112]:
check_same(quantity, quantity_group)

quantity has 5 unique values 

quantity_group has 5 unique values 

0 elements in quantity or quantity_group but not both: 
 set()
100.00% of elements are the same


In [113]:
check_same(source, source_type)

source has 10 unique values 

source_type has 7 unique values 

7 elements in source or source_type but not both: 
 {'river/lake', 'river', 'machine dbh', 'unknown', 'lake', 'hand dtw', 'borehole'}
99.62% of elements are the same


In [114]:
check_same(wpt, wptg)

waterpoint_type has 7 unique values 

waterpoint_type_group has 6 unique values 

1 elements in waterpoint_type or waterpoint_type_group but not both: 
 {'communal standpipe multiple'}
99.90% of elements are the same


In [86]:
train_values['recorded_by'].unique()

array(['GeoData Consultants Ltd'], dtype=object)

In [96]:
# Number of unique values for each column [.unique vs .value_counts]
len_uniques = [len(train_values[i].unique()) for i in train_values.columns]
pd.DataFr

[59400,
 98,
 356,
 1898,
 2428,
 2146,
 57516,
 57517,
 37400,
 65,
 9,
 19288,
 21,
 27,
 20,
 125,
 2092,
 1049,
 3,
 1,
 13,
 2697,
 3,
 55,
 18,
 13,
 7,
 12,
 5,
 7,
 7,
 8,
 6,
 5,
 5,
 10,
 7,
 3,
 7,
 6]